In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

D:\LLAS\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/",one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [3]:
total_epoch = 200
batch_size = 100
learning_rate = 0.01
n_hidden = 256
n_input = 28*28
n_noise = 128
n_class = 10

In [4]:
#Real data
X = tf.placeholder(tf.float32,[None,n_input])
#Noise data
Z = tf.placeholder(tf.float32,[None,n_noise])
#Y Hint Data
Y = tf.placeholder(tf.float32,[None,n_class])

In [5]:
def generater(noise, labels):
    with tf.variable_scope('Generater'):
        inputs = tf.concat([noise, labels],1)
        hidden = tf.layers.dense(inputs,n_hidden,tf.nn.relu)
        output = tf.layers.dense(hidden,n_input,activation=tf.nn.sigmoid)
        
        return output

In [6]:
def discriminaster(inputs, labels, reuse=None):
    with tf.variable_scope('Discriminaster') as scope:
        if(reuse):
            scope.reuse_variables()
            
        inputs = tf.concat([inputs, labels], 1)
        hidden = tf.layers.dense(inputs, n_hidden, activation=tf.nn.relu)
        output = tf.layers.dense(hidden , 1, activation=None)
            
        return output

In [7]:
def get_noise(batch_size, n_noise):
    return np.random.uniform(-1., 1., size = [batch_size, n_noise])

In [8]:
G = generater(Z, Y)

In [9]:
D_real = discriminaster(X, Y)
D_gene = discriminaster(G,Y,True)

In [10]:
loss_D_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real,labels=tf.ones_like(D_real)))

loss_D_gene = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene,labels=tf.zeros_like(D_gene)))

In [11]:
loss_D = loss_D_gene + loss_D_real

In [12]:
loss_G = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_gene, labels=tf.ones_like(D_gene)))

In [13]:
vars_D = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='Discriminaster')
vars_G = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='Generater')
